In [1]:
import spacy
import numpy as np
import itertools
import lemmatizer 
import datetime

C:\Users\melis\anaconda3\envs\nlp\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
from vocabulary import *

In [2]:
ner_model = spacy.load('model-best')

C:\Users\melis\anaconda3\envs\nlp\lib\site-packages\spacy\util.py:877: UserWarning: [W095] Model 'tr_pipeline' (0.0.0) was trained with spaCy v3.3 and may not be 100% compatible with the current version (3.4.2). If you see errors or degraded performance, download a newer compatible model or retrain your custom model with the current spaCy version. For more details and available updates, run: python -m spacy validate
  warnings.warn(warn_msg)


In [88]:
doc = ner_model("Ankara trenine bilet 2 yetişkin")

colors = {"DURAK": "#F67DE3", "YOLCU": "#7DF6D9", "SAYI":"#a6e22d", "AY":"#FF5733", "GÜN":"#2D14FF"}
options = {"colors": colors} 

spacy.displacy.render(doc, style="ent", options= options, jupyter=True)

In [125]:
class Compare:
    
    def compare_dates(self, response_content):
        date0 = response_content['date0']
        date1 = response_content['date1']
        if date0 != "" and date1 != "":
            if date0 < date1:
                temp_date = date0
                date0 = date0
                date1 = temp_date
                
        response_content['date0'] = date0
        response_content['date1'] = date1
        
        return response_content
            
    def compare_weekdays(self, weekdays):
        
        if len(weekdays) == 2 and weekdays[1] < weekdays[0]:
            weekdays[1] = weekday[1] + datetime.timedelta(weeks=1)
        return weekdays
    
    def compare_delays(self, delays, sdelays):
        if len(delays)==1 and len(sdelays)==1:
            delay = delays[0]
            sdelay = sdelays[0]
            # Compare delay indexes Example: delay = ('haftaya',2) sdelay = (2,'gün sonraya',3)
            #                                                   ^            ^               ^
            #                                                 index     sdelay amount      index
            if delay[1] < sdelay[2]:
                # date0 = format_delays(delay)
                # date1 = date0 + format_sdelays(sdelay)
                
                delay_first = True
            else:
                # date0 = today + format_sdelays(sdelay)
                # date1 = today + format_delays(delay)
                delay_first = False
            return delay_first

In [79]:
class Format(Compare):
    
    def format_date_string(self, date_list):
        formatted_date_string = [str(date.strftime("%d-%m-%Y")) for date in date_list]
        return formatted_date_string
    
    def format_weekday(self, weekday_index):
        today = datetime.date.today()
        weekdays = [(today + datetime.timedelta((index-today.weekday()) % 7)) for index in weekday_index]
        weekdays = self.compare_weekdays(weekdays)
        return weekdays
    
    def format_delay(self, delay):
        today = datetime.date.today()
        dates = [(today + datetime.timedelta(weeks=1)) for d in delay]
        return dates
    
    def format_sdelay(self, sdelay):
        today = datetime.date.today()
        dates = []
        #delay tagleri içerisinde geçen kelimeye göre hafta veya gün sonrasını alma
        for sd in sdelay:
            if 'hafta' in sd[1]:
                dates.append(datetime.timedelta(weeks=int(sd[0]))
            if 'gün' in sd[1]:
                dates.append(datetime.timedelta(days=int(sd[0]))
                
        return date
    
    def format_delay_indexes(self, delays, sdelays):
        delay_first = self.compare_delays(delays, sdelays):
        if delay_first:
            formatted_delay = self.format_delay(delays)
            formatted_sdelay = formatted_delay + datetime.timedelta(sdelays)
    
    def format_datetime(self, date):
        month_list = ['Ocak','Şubat','Mart','Nisan','Mayıs','Haziran','Temmuz','Ağustos','Eylül','Ekim','Kasım','Aralık']
        current_year = datetime.date.today().year
        formatted_date = datetime.datetime(current_year, month_list.index(str(date[1])) + 1, int(date[0]))
        return formatted_date
    
    def format_dates(self, dates):
        
        formatted_dates = [self.format_datetime(date) for date in dates]
        
        if len(formatted_dates)==2 and formatted_dates[0] > formatted_dates[1]: 
            formatted_dates[1] = formatted_dates[1] + datetime.timedelta(days=365)
                
        return formatted_dates

In [121]:
class Process(Format):
    
    def __init__(self):
        self.passenger_vocab = {"ADT":["yolcu", "yetişkin", "kişi","kendime","kendim"],
                                "CHD":["çocuk","çocuğum"],
                                "PET":["hayvan"],
                                "YNG":["öğrenci", "genç"],
                                "TCH":["öğretmen", "öğretim görevlisi"],
                                "PRS":['basın', 'muhabir'],
                                "MLT":['asker'],
                                "STF":['personel', 'çalışan'],
                                "60Y":['60 yaş'],
                                "65+":['65 yaş']}
        self.station_vocab = {"Söğütlüçeşme":"SGTC",
                              "Ankara":"ANKR",
                              "İstanbul":"ISTN",
                              "Eskişehir":"ESKR",
                              "Polatlı":"PLTL"}
    
    def process_date_indexes(self, months, numbers):
        
        dates = []
        for month in months:
            for number in numbers:
                if month[1]-1==number[1]:
                    dates.append((number[0],month[0]))
                    numbers.remove(number)
                elif month[1]+1==number[1]:
                    dates.append((number[0],month[0]))
                    numbers.remove(number)
                    
        dates = self.format_dates(dates)
        return dates, numbers

    def process_passenger_indexes(self, passenger_types, numbers):
        
        passengers = []
        for passenger in passenger_types:
            for number in numbers:
                if passenger[1]-1==number[1]:
                    passengers.append((number[0],passenger[0]))
                    numbers.remove(number)
                else:
                    passengers.append((1,passenger[0]))
        return passengers
    
    def process_sdelay_indexes(self, sdelays, numbers):
        
        s_delays = []
        for delay in sdelays:
            for number in numbers:
                if delay[1]-1==number[1]:
                    s_delays.append((number[0],delay[0],delay[1]))
                    numbers.remove(number)
                    
        s_delays = self.format_sdelay(s_delays)
        return s_delays, numbers
    
    def process_indexes(self, dates, passengers, sdelays, numbers):
        
        if len(dates) != 0:
            dates, numbers = self.process_date_indexes(dates, numbers)
        if len(sdelays) != 0:
            sdelays, numbers = self.process_sdelay_indexes(sdelays, numbers)
        if len(passengers) != 0:
            passengers = self.process_passenger_indexes(passengers, numbers)
        
        return dates, passengers, sdelays
    
    def process_dates(self, dates, weekdays):
        
        if len(weekdays) != 0:
            weekdays = self.format_weekday(weekdays)
        
        if len(dates)==2:
            self.response_content['date0'] = dates[0]
            self.response_content['date1'] = dates[1]
        elif len(dates)==1 and len(weekdays)==0:
            self.response_content['date0'] = dates[0]
        elif len(dates)==1 and len(weekdays)==1:
            self.response_content['date0'] = weekdays[0]
            self.response_content['date1'] = dates[1]
        elif len(weekdays)==2:
            self.response_content['date0'] = weekdays[0]
            self.response_content['date1'] = weekdays[1]    
        else:
            tomorrow = [datetime.date.today() + datetime.timedelta(days=1)]
            tomorrow = self.format_date_string(tomorrow)
            self.response_content['date0'] = tomorrow[0]
        return self.response_content
    
    def process_delays(self, delays, sdelays):

        if len(delays) != 0:
            delays = self.format_delay(delays)
            
        if len(delays)==2:
            self.response_content['date0'] = delays[0]
            self.response_content['date1'] = delays[1]
        elif len(delays)==1 and len(sdelays)==0:
            self.response_content['date0'] = delays[0]
        elif len(delays)==1 and len(sdelays)==1:
            self.response_content['date0'] = delays[0]
            self.response_content['date1'] = sdelays[0]
        elif len(sdelays)==2:
            self.response_content['date0'] = sdelays[0]
            self.response_content['date1'] = sdelays[1]
        
        return self.response_content
    
    def process_stations(self, stations, defaultLocation):
        
        if len(stations) == 2:
            From, To = stations
            self.response_content['from0'] = From
            self.response_content['to0'] = To
            
            self.response_content['from1'] = To
            self.response_content['to1'] = From
            
        elif len(stations) == 1: 
            
            # Assign default_location as the departure location that is retrieved from0 GPS
            From, To = defaultLocation, stations[0]

            self.response_content['from0'] = From
            self.response_content['to0'] = To
            
            self.response_content['from1'] = To
            self.response_content['to1'] = From
            
        elif len(stations) == 0:
            self.response_content['no_stations'] = True
            
        return self.response_content
                    
    def process_passengers(self, passengers):
        
        self.response_content['Passengers'] = passengers
        return self.response_content

    def process_self(self, self_):
        
        if len(self_) != 0:
            self.response_content['Passengers'].append((1, self_[0]))
        return self.response_content
 
    def process_station_tags(self, response_content):

        if response_content['from0'] in self.station_vocab: response_content['from0'] = self.station_vocab[response_content['from0']]

        if response_content['from1'] in self.station_vocab: response_content['from1'] = self.station_vocab[response_content['from1']]

        if response_content['to0'] in self.station_vocab: response_content['to0'] = self.station_vocab[response_content['to0']]

        if response_content['to1'] in self.station_vocab: response_content['to1'] = self.station_vocab[response_content['to1']]

        if response_content['date1'] == "":
            del response_content['date1']
            del response_content['from1']
            del response_content['to1']
                
        return response_content

    def process_passenger_tags(self, response_content):
        
        passenger_types = {}
        for passenger in response_content['Passengers']:
            for tag, types in self.passenger_vocab.items(): 
                #yolcu tipinin bulunduğu liste
                if passenger[1] in types:
                    passenger_tag = tag
                    passenger_count = passenger[0]
                    #eğer yolcu tagi listede varsa yolcu sayısını üstüne ekle
                    if passenger_tag in passenger_types.keys():
                        passenger_types[passenger_tag] += passenger_count
                    #yeni bir tagse yeni yolcu ekle
                    else:
                        passenger_types[passenger_tag] = passenger_count
                        
        del response_content['Passengers']
        
        for key, value in passenger_types.items():
            response_content[key] = str(value)
            
        return response_content
    
    def process_available_stations(self, response_content, response):
        
        if response_content['no_stations']:
            response['exampleArrival']['stationList'] = list(self.station_vocab.keys())
            
            
        del response_content['no_stations']
        
        if response_content['from0'] == response_content['to0']:
            response['exampleArrival']['stationList'] = list(self.station_vocab.keys())
            
        return response_content, response
    
    def process_url(self, response):
        if response['exampleArrival']["stationList"] != [""]:
            response['url'] = ""
        return response

In [81]:
haftaya kalkıp 2 gün sonra
0, 2
2 gün sonra kalkıp haftaya dönen
0, 4

SyntaxError: invalid syntax (4190692089.py, line 1)

In [117]:
class Assemble(Process):
    
    def __init__(self):
        
        super().__init__()

        self.response = {}
        
        #self.response['exampleDeparture'] = {}
        self.response['exampleArrival'] = {}
        
        #self.response['exampleDeparture']["message"] = "Lütfen gitmek istediğiniz durağı belirtin."
        #self.response['exampleDeparture']["stationList"] = [""]
        
        self.response['exampleArrival']["message"] = "Lütfen geçerli bir durak belirtiniz."
        self.response['exampleArrival']["stationList"] = [""]
        
        self.response['url'] = ""
    
    def assemble_url(self): 
    
        
        self.response_content = self.process_station_tags(self.response_content)
        self.response_content = self.process_passenger_tags(self.response_content)

        link = "/availability?"

        for key, value in self.response_content.items():
            link = link+key+"="+value+"&"
            
        #linkteki son & işaretini alma
        link = link[:-1]
        self.response['url'] = link 
        return self.response

In [93]:
class Apply:
    
    def apply_ner_model(self, text):
        ner_model = spacy.load('model-best')
        doc = ner_model(text)
        return doc
    
    def apply_lemmatization_model(self, word):
        lemmatized_word = lemmatizer.lemmatizeWord(word)
        return lemmatized_word

In [9]:
import time

In [113]:
text = "Eskişehir trenine 2 bilet"

request_ = {"defaultLocation": "Ankara", 
            "text": f"{text}"} 

In [114]:
# datetime karşılaştır
# lemmatize
# format class
# yeni model
# vocabulary.py
# architecture
# büyük harf +

In [115]:
s = time.time()
extract = Extract()
response = extract.extract(request_)
e = time.time()
print(e-s)
response

[(2, 1)]
0.21130132675170898


{'exampleArrival': {'message': 'Lütfen geçerli bir durak belirtiniz.',
  'stationList': ['']},
 'url': '/availability?date0=03-12-2022&from0=ANKR&to0=ESKR'}

In [120]:
class Extract(Assemble, Apply):
    """
    Extract information from the received request and return a url as a response
    """
    def __init__(self):
        
        super().__init__()
        self.station_list = ['Ankara', 'İstanbul', 'Eskişehir', 'İzmir', 'Kars', 'Konya', 'Malatya', 'Adana', 'Polatlı', 'Söğütlüçeşme', 'Eryaman', 'Bakırköy', 'Bozüyük', 'Sakarya']
        self.month_list = ['Ocak', 'Şubat', 'Mart', 'Nisan', 'Mayıs', 'Haziran', 'Temmuz', 'Ağustos', 'Eylül', 'Ekim', 'Kasım', 'Aralık']
        self.weekday_list = ['pazartesi', 'salı', 'çarşamba', 'perşembe', 'cuma', 'cumartesi', 'pazar']
        self.self_list = ['ben', 'bana', 'benim adıma', 'kendim', 'kendime']
        self.s_delay_list = ['güne', 'gün sonraya', 'gün sonrasına', 'gün ilerisine', 'ay', 'hafta', 'ayın', 'haftasına']
        self.gender_list = ['erkek', 'kadın', 'kız', 'hanımefendi', 'beyefendi']
        self.delay_list = ['haftaya', 'günübirlik', 'haftaya yarın', 'ertesi', 'ertesi gün', 'ertesi güne', 'yarın', 'yarına', 'yarın için', 'bugün', 'bugüne', 'haftaya bugün', 'sabaha', 'akşama', 'akşam için', 'sabah için', 'öğleye', 'öğlene', 'öğleden sonraya', 'öğleden sonrasına', 'hafta içi', 'hafta sonu']
        self.passenger_list = ['yolcu', 'engelli', 'arkadaş', 'askeri personel', 'sakat', 'gazi', 'kişilik', 'kişiye', 'şehit yakını', 'gazi yakını', 'hamile', 'çocuklu', 'arkadaşım', 'veli', 'yetişkin', 'çocuğum', 'yaşlı', 'kişi', 'çocuk', 'hayvan', 'öğrenci', 'genç', 'öğretmen', 'öğretim görevlisi', 'basın', 'muhabir', 'asker', 'personel', 'çalışan', '60 yaş', '65 yaş']
        self.number_list = list(np.arange(1,32).astype(str))
        
        self.response_content = {}
        
        self.response_content['date0'] = ""
        self.response_content['date1'] = ""

        self.response_content['from0'] = ""
        self.response_content['to0'] = ""
        
        self.response_content['from1'] = ""
        self.response_content['to1'] = ""
        
        self.response_content['Passengers'] = []
        
        self.response_content['no_stations'] = False
        
        self.entity_list = {'DURAK':self.station_list,
                            'AY':self.month_list,
                            'YOLCU':self.passenger_list,
                            'GÜN':self.weekday_list,
                            'SELF':self.self_list,
                            'DELAY':self.delay_list,
                            'SDELAY':self.s_delay_list,
                            'CİNSİYET':self.gender_list,
                            'SAYI':self.number_list}
        
    def extract_entities(self, doc):
        
        stations = []
        months = []
        passenger_types = []
        weekdays = []
        self_ = []
        delays = []
        s_delays = []
        genders = []
        numbers = []
        
        for index, ent in enumerate(doc.ents):
            label = ent.label_
            ent_list = self.entity_list[label]
            
            if label == 'DURAK' or label == 'AY' or label == 'GÜN' or label == 'SAYI':
                ent = self.apply_lemmatization_model(str(ent))
                
            if label== 'YOLCU':
                ent = str(ent).split(',')[0]
                if " " not in str(ent):
                    ent = self.apply_lemmatization_model(str(ent))

            if label != 'DURAK' and label != 'AY':
                ent = str(ent).lower()
                
            if str(ent) in ent_list:
                if label == 'DURAK': stations.append(str(ent)) 

                if label == 'AY': months.append((str(ent),index))

                if label == 'YOLCU': passenger_types.append((str(ent),index)) 

                if label == 'GÜN': weekdays.append(str(ent)) 

                if label == 'SELF': self_.append(str(ent)) 
                
                if label == 'DELAY': delays.append((str(ent),index))

                if label == 'SDELAY': s_delays.append((str(ent),index))
                
                if label == 'CİNSİYET': genders.append((str(ent),index)) 
                
                if label == 'SAYI': numbers.append((int(str(ent)),index))   
                
        dates, passengers, sdelays = self.process_indexes(months, passenger_types, s_delays, numbers)
        entities = {'station':stations, 
                    'dates':dates, 
                    'weekdays':weekdays, 
                    'delays':delays,
                    'sdelays':sdelays, 
                    'passengers':passengers, 
                    'self':self_, 
                    'genders':genders}
        return entities
                
    def extract_stations(self, entities, defaultLocation):
        
        stations = entities['station']
        self.response_content = self.process_stations(stations, defaultLocation)
        return self.response_content
            
    def extract_dates(self, entities):
        
        dates = entities['dates']
        weekdays = [self.weekday_list.index(weekday) for weekday in entities['weekdays']]
        delays = entities['delays']
        sdelays = entities['sdelays']
        
        self.response_content = self.process_dates(dates, weekdays)
        self.response_content = self.process_delays(delays, sdelays)
        return self.response_content
    
    def extract_passengers(self, entities):
        
        passengers = entities['passengers']
        self_ = entities['self']
        genders = entities['genders']
        
        self.response_content = self.process_passengers(passengers)
        self.response_content = self.process_self(self_)
        return self.response_content
        
    def extract(self, request_): 
        
        text = request_['text']
        defaultLocation =  request_['defaultLocation']
        
        doc = self.apply_ner_model(text)
        entities = self.extract_entities(doc)
        
        self.response_content = self.extract_dates(entities)
        self.response_content = self.extract_stations(entities, defaultLocation)
        self.response_content = self.extract_passengers(entities)
        
        self.response_content = self.compare_dates(self.response_content)
        self.response_content, self.response = self.process_available_stations(self.response_content, self.response)
        self.response = self.assemble_url()
        self.response = self.process_url(self.response)
        
        return self.response

In [ ]:
class Process(Format):
    
    def __init__(self):
        self.passenger_vocab = {"ADT":["yolcu", "yetişkin", "kişi","kendime","kendim"],
                                "CHD":["çocuk","çocuğum"],
                                "PET":["hayvan"],
                                "YNG":["öğrenci", "genç"],
                                "TCH":["öğretmen", "öğretim görevlisi"],
                                "PRS":['basın', 'muhabir'],
                                "MLT":['asker'],
                                "STF":['personel', 'çalışan'],
                                "60Y":['60 yaş'],
                                "65+":['65 yaş']}
        self.station_vocab = {"Söğütlüçeşme":"SGTC",
                              "Ankara":"ANKR",
                              "İstanbul":"ISTN",
                              "Eskişehir":"ESKR",
                              "Polatlı":"PLTL"}
        
    def process_dates(self, dates, weekdays):
        
        if len(weekdays) != 0:
            weekdays = self.format_weekday(weekdays)
        
        if len(dates)==2:
            self.response_content['date0'] = dates[0]
            self.response_content['date1'] = dates[1]
        elif len(dates)==1 and len(weekdays)==0:
            self.response_content['date0'] = dates[0]
        elif len(dates)==1 and len(weekdays)==1:
            self.response_content['date0'] = weekdays[0]
            self.response_content['date1'] = dates[1]
        elif len(weekdays)==2:
            self.response_content['date0'] = weekdays[0]
            self.response_content['date1'] = weekdays[1]    
        else:
            tomorrow = [datetime.date.today() + datetime.timedelta(days=1)]
            tomorrow = self.format_date_string(tomorrow)
            self.response_content['date0'] = tomorrow[0]
        return self.response_content
    
    def process_delays(self, delays, sdelays):

        if len(delays) != 0:
            delays = self.format_delay(delays)
            
        if len(delays)==2:
            self.response_content['date0'] = delays[0]
            self.response_content['date1'] = delays[1]
        elif len(delays)==1 and len(sdelays)==0:
            self.response_content['date0'] = delays[0]
        elif len(delays)==1 and len(sdelays)==1:
            self.response_content['date0'] = delays[0]
            self.response_content['date1'] = sdelays[0]
        elif len(sdelays)==2:
            self.response_content['date0'] = sdelays[0]
            self.response_content['date1'] = sdelays[1]
        
        return self.response_content
    
    def process_stations(self, stations, defaultLocation):
        
        if len(stations) == 2:
            From, To = stations
            self.response_content['from0'] = From
            self.response_content['to0'] = To
            
            self.response_content['from1'] = To
            self.response_content['to1'] = From
            
        elif len(stations) == 1: 
            
            # Assign default_location as the departure location that is retrieved from0 GPS
            From, To = defaultLocation, stations[0]

            self.response_content['from0'] = From
            self.response_content['to0'] = To
            
            self.response_content['from1'] = To
            self.response_content['to1'] = From
            
        elif len(stations) == 0:
            self.response_content['no_stations'] = True
            
        return self.response_content
                    
    def process_passengers(self, passengers):
        
        self.response_content['Passengers'] = passengers
        return self.response_content

    def process_self(self, self_):
        
        if len(self_) != 0:
            self.response_content['Passengers'].append((1, self_[0]))
        return self.response_content
 
    def process_station_tags(self, response_content):

        if response_content['from0'] in self.station_vocab: response_content['from0'] = self.station_vocab[response_content['from0']]

        if response_content['from1'] in self.station_vocab: response_content['from1'] = self.station_vocab[response_content['from1']]

        if response_content['to0'] in self.station_vocab: response_content['to0'] = self.station_vocab[response_content['to0']]

        if response_content['to1'] in self.station_vocab: response_content['to1'] = self.station_vocab[response_content['to1']]

        if response_content['date1'] == "":
            del response_content['date1']
            del response_content['from1']
            del response_content['to1']
                
        return response_content

    def process_passenger_tags(self, response_content):
        
        passenger_types = {}
        for passenger in response_content['Passengers']:
            for tag, types in self.passenger_vocab.items(): 
                #yolcu tipinin bulunduğu liste
                if passenger[1] in types:
                    passenger_tag = tag
                    passenger_count = passenger[0]
                    #eğer yolcu tagi listede varsa yolcu sayısını üstüne ekle
                    if passenger_tag in passenger_types.keys():
                        passenger_types[passenger_tag] += passenger_count
                    #yeni bir tagse yeni yolcu ekle
                    else:
                        passenger_types[passenger_tag] = passenger_count
                        
        del response_content['Passengers']
        
        for key, value in passenger_types.items():
            response_content[key] = str(value)
            
        return response_content
    
    def process_available_stations(self, response_content, response):
        
        if response_content['no_stations']:
            response['exampleArrival']['stationList'] = list(self.station_vocab.keys())
            
        if response_content['from0'] == response_content['to0']:
            response['exampleArrival']['stationList'] = list(self.station_vocab.keys())
            
        del response_content['no_stations']
            
        return response_content, response
    
    def process_url(self, response):
        if response['exampleArrival']["stationList"] != [""]:
            response['url'] = ""
        return response

In [ ]:
self.today
Obtain() (indexes, sdelays-delays, etc.)
Transform() (tags)
compare_dates() # en son
vocabulary